In [1]:
import json
import os
import random
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, InputExample, evaluation

class EvidenceClaimRetriever:
    def __init__(self, model_name='all-mpnet-base-v2', batch_size=16, num_epochs=3):
        self.model = SentenceTransformer(model_name)
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.evidence_map = {}
        self.label_mapping = {
            'SUPPORTS': 1,
            'REFUTES': 1,
            'DISPUTED': 1,
            'NOT_ENOUGH_INFO': 0
        }

    def load_data(self, claims_path, evidence_path):
        """Load training data and evidence corpus"""
        # Load evidence corpus
        with open(evidence_path) as f:
            self.evidence_map = json.load(f)

        # Load training claims
        with open(claims_path) as f:
            train_claims = json.load(f)

        # Create training examples
        self.train_examples = []
        for claim_id, claim_data in train_claims.items():
            self._process_claim(claim_data)

    def _process_claim(self, claim_data):
        """Create training examples for a single claim"""
        claim_text = claim_data['claim_text']
        evidence_ids = claim_data['evidences']

        # Add positive examples
        for eid in evidence_ids:
            if eid in self.evidence_map:
                self.train_examples.append(InputExample(
                    texts=[claim_text, self.evidence_map[eid]],
                    label=self.label_mapping[claim_data['claim_label']]
                ))

        # Add negative examples
        self._add_negative_examples(claim_text, evidence_ids, claim_data['claim_label'])

    def _add_negative_examples(self, claim_text, positive_ids, label):
        """Add negative examples with proper ratio"""
        all_evidence_ids = list(self.evidence_map.keys())
        negative_candidates = list(set(all_evidence_ids) - set(positive_ids))

        # Determine number of negatives based on label
        if label == 'NOT_ENOUGH_INFO':
            num_negatives = min(2, len(negative_candidates))
        else:
            num_negatives = min(3 * len(positive_ids), len(negative_candidates))

        # Add selected negatives
        for eid in random.sample(negative_candidates, num_negatives):
            self.train_examples.append(InputExample(
                texts=[claim_text, self.evidence_map[eid]],
                label=0
            ))

    def load_dev_data(self, dev_claims_path):
        """Load separate dev set for validation"""
        with open(dev_claims_path) as f:
            dev_claims = json.load(f)

        # Prepare for InformationRetrievalEvaluator
        self.dev_queries = {}
        self.relevant_docs = {}

        for claim_id, claim_data in dev_claims.items():
            claim_text = claim_data['claim_text']
            evidence_ids = claim_data['evidences']

            self.dev_queries[claim_id] = claim_text
            self.relevant_docs[claim_id] = set(evidence_ids)

    def train(self, output_dir='./trained_model'):
        """Train using dedicated dev set for validation"""
        # Create evaluator using separate dev set
        dev_evaluator = evaluation.InformationRetrievalEvaluator(
            queries=self.dev_queries,
            corpus=self.evidence_map,
            relevant_docs=self.relevant_docs,
            show_progress_bar=True
        )

        # Create DataLoader directly from InputExamples
        train_dataloader = DataLoader(
            self.train_examples,  # Use the list of InputExamples directly
            shuffle=True,
            batch_size=self.batch_size
        )

        # Use MultipleNegativesRankingLoss
        train_loss = losses.MultipleNegativesRankingLoss(self.model)

        # Training configuration
        self.model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            evaluator=dev_evaluator,
            epochs=self.num_epochs,
            evaluation_steps=100,
            warmup_steps=int(len(self.train_examples) * 0.1),
            output_path=output_dir,
            show_progress_bar=True
        )
        
    def retrieve(self, claim_text, top_k=5):
        """Retrieve evidence for a claim"""
        claim_embed = self.model.encode(claim_text)
        scores = torch.nn.functional.cosine_similarity(
            torch.tensor(claim_embed),
            torch.tensor(self.evidence_embeddings)
        )
        top_indices = torch.topk(scores, k=top_k).indices.tolist()
        return [(self.evidence_ids[i], scores[i].item()) for i in top_indices]

    def index_evidence(self, save_path='data/evidence_embeddings.pth'):
        """Precompute all evidence embeddings and optionally save them"""
        self.evidence_ids = list(self.evidence_map.keys())
        self.evidence_texts = [self.evidence_map[eid] for eid in self.evidence_ids]

        # Check if embeddings already exist
        if save_path and os.path.exists(save_path):
            print("Loading precomputed evidence embeddings...")
            data = torch.load(save_path, weights_only=False)
            self.evidence_embeddings = data['embeddings']
            self.evidence_ids = data['ids']
        else:
            print("Computing evidence embeddings...")
            self.evidence_embeddings = self.model.encode(
                self.evidence_texts,
                show_progress_bar=True,
                batch_size=32
            )
            # Save embeddings if save_path is provided
            if save_path:
                print("Saving evidence embeddings...")
                torch.save({'embeddings': self.evidence_embeddings, 'ids': self.evidence_ids}, save_path)


    

In [2]:
evidenceRetriever = EvidenceClaimRetriever(
        model_name='all-MiniLM-L6-v2',
        batch_size=32,
        num_epochs=5
    )


In [3]:

# 1. Load training data and evidence
evidenceRetriever.load_data(
    claims_path='data/train-claims.json',
    evidence_path='data/evidence.json'
)


In [4]:

# 2. Load separate dev set
evidenceRetriever.load_dev_data('data/dev-claims-baseline.json')

In [5]:

# 3. Precompute evidence embeddings
evidenceRetriever.index_evidence()

Loading precomputed evidence embeddings...


In [9]:


# 4. Train with dev set validation
# trainer.train(output_dir='./climate_retriever')


/Users/jeffphan8816/Desktop/StudyUnimelb/NLP/NLP_a3_gp40/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
100,No log,No log,0.240260,0.396104,0.448052,0.538961,0.240260,0.164502,0.123377,0.083766,0.100758,0.188961,0.224567,0.287987,0.248052,0.331027,0.204947
200,No log,No log,0.201299,0.324675,0.389610,0.500000,0.201299,0.138528,0.107792,0.078571,0.082576,0.159848,0.206710,0.280411,0.224574,0.281705,0.173821
300,No log,No log,0.194805,0.337662,0.396104,0.467532,0.194805,0.145022,0.112987,0.074675,0.081277,0.169913,0.209632,0.269372,0.220480,0.277394,0.173043
359,No log,No log,0.220779,0.350649,0.422078,0.506494,0.220779,0.147186,0.122078,0.079870,0.091017,0.173485,0.226299,0.287013,0.237954,0.305148,0.186771
400,No log,No log,0.227273,0.363636,0.428571,0.512987,0.227273,0.151515,0.123377,0.081818,0.101840,0.178030,0.227597,0.294264,0.245545,0.310774,0.194059
500,3.750600,No log,0.220779,0.389610,0.448052,0.564935,0.220779,0.166667,0.131169,0.089610,0.095671,0.190693,0.237771,0.323918,0.259840,0.325193,0.201078
600,3.750600,No log,0.227273,0.402597,0.461039,0.597403,0.227273,0.168831,0.135065,0.095455,0.094589,0.193182,0.249134,0.333009,0.266805,0.335225,0.207562
700,3.750600,No log,0.240260,0.409091,0.493506,0.616883,0.240260,0.170996,0.144156,0.094805,0.102381,0.193939,0.260065,0.338853,0.273624,0.349052,0.216047
718,3.750600,No log,0.240260,0.383117,0.493506,0.610390,0.240260,0.158009,0.146753,0.096104,0.098052,0.183333,0.268182,0.340909,0.271977,0.345552,0.212218
800,3.750600,No log,0.246753,0.396104,0.525974,0.616883,0.246753,0.160173,0.153247,0.097403,0.099351,0.185173,0.285931,0.349242,0.278150,0.355677,0.216744


Corpus Chunks: 100%|██████████| 25/25 [08:36<00:00, 20.66s/it]


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

class LLaMAFilter:
    def __init__(self, model_name="meta-llama/Llama-3.1-8B-Instruct"):
        self.device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name,
                                                          torch_dtype=torch.float16,
                                                          ).to(self.device)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
    def filter_evidence(self, claim, evidence_candidates, final_k=5):
        """Filter evidence using LLaMA's relevance scoring"""
        prompts = [
            f"Claim: {claim}\nEvidence: {text}\nRate relevance 1-5 (1 is not relevant, 5 is perfect match): "
            for eid, text in evidence_candidates
        ]
            
        inputs = self.tokenizer(prompts, return_tensors="pt", padding=True).to(self.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=2,
            temperature=0.1,
            pad_token_id=self.tokenizer.eos_token_id
        )
        scored_evidence = []
        for (eid, text), output in zip(evidence_candidates, outputs):
            output_text = self.tokenizer.decode(output, skip_special_tokens=True)
            # Extract the first digit from the output
            digits = re.findall(r'\d', output_text)
            if digits:
                rating = int(digits[0])
                # Ensure rating is between 1-5
                rating = max(1, min(5, rating))
            else:
                # Default to middle rating if no digit found
                rating = 3
            scored_evidence.append((eid, text, rating))
        scored_evidence.sort(key=lambda x: x[2], reverse=True)
        return scored_evidence[:final_k]

In [14]:
def get_evidence_text(evidences):
    return [(eid, evidenceRetriever.evidence_map[eid]) for eid in evidences]

In [8]:
def initial_retrieval(retriever, test_claims_path):
    with open(test_claims_path) as f:
        test_claims = json.load(f)
    
    output = {}
    for claim_id, claim_data in test_claims.items():
        # Retrieve candidates
        candidates = retriever.retrieve(claim_data['claim_text'], top_k=10)
        retrieved = [eid for eid, _  in candidates[:10]]
        output[claim_id] = {
            "initial_evidences": retrieved,
            "claim_text": claim_data['claim_text']
        }

    # store output
    with open('data/initial_retrieval_output.json', 'w') as f:
        json.dump(output, f, indent=2)
    return output


In [9]:
res = initial_retrieval(evidenceRetriever, test_claims_path='data/dev-claims-baseline.json')

In [10]:
llama_filter = LLaMAFilter()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
def process_claims(llama_filterer, dev_claims_set):
    for claim_id, claim_data in dev_claims_set.items():
        candidates = claim_data['initial_evidences']
        print(candidates)
        filtered = llama_filterer.filter_evidence(claim_data['claim_text'], get_evidence_text(candidates))
        dev_claims_set[claim_id]['filtered_evidences'] = [eid for eid, _, _ in filtered]
    
        # store output
    with open('data/filtered_retrieval_output.json', 'w') as f:
        json.dump(dev_claims_set, f, indent=2)
    return dev_claims_set

res = process_claims(llama_filter,res)
res

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['evidence-67732', 'evidence-572512', 'evidence-780332', 'evidence-1061888', 'evidence-452156', 'evidence-685728', 'evidence-48256', 'evidence-1002110', 'evidence-1050894', 'evidence-1075134']


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['evidence-647121', 'evidence-559290', 'evidence-415619', 'evidence-361694', 'evidence-949910', 'evidence-970402', 'evidence-121187', 'evidence-866495', 'evidence-1075865', 'evidence-419267']


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['evidence-526112', 'evidence-366949', 'evidence-510347', 'evidence-488270', 'evidence-475230', 'evidence-980959', 'evidence-107116', 'evidence-529355', 'evidence-526003', 'evidence-1056020']


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['evidence-1175280', 'evidence-1050619', 'evidence-754191', 'evidence-930563', 'evidence-748548', 'evidence-326643', 'evidence-608016', 'evidence-1196738', 'evidence-877068', 'evidence-900586']


{'claim-752': {'initial_evidences': ['evidence-67732',
   'evidence-572512',
   'evidence-780332',
   'evidence-1061888',
   'evidence-452156',
   'evidence-685728',
   'evidence-48256',
   'evidence-1002110',
   'evidence-1050894',
   'evidence-1075134'],
  'claim_text': '[South Australia] has the most expensive electricity in the world.',
  'filtered_evidences': ['evidence-1075134',
   'evidence-67732',
   'evidence-572512',
   'evidence-780332',
   'evidence-1061888']},
 'claim-375': {'initial_evidences': ['evidence-647121',
   'evidence-559290',
   'evidence-415619',
   'evidence-361694',
   'evidence-949910',
   'evidence-970402',
   'evidence-121187',
   'evidence-866495',
   'evidence-1075865',
   'evidence-419267'],
  'claim_text': 'when 3 per cent of total annual global emissions of carbon dioxide are from humans and Australia prod\xaduces 1.3 per cent of this 3 per cent, then no amount of emissions reductio\xadn here will have any effect on global climate.',
  'filtered_evide

In [ ]:
class HybridRetriever:
    def __init__(self, retriever: EvidenceClaimRetriever, llama_filter: LLaMAFilter):
        self.retriever = retriever
        self.filter = llama_filter

    def load_data(self, claims_path, evidence_path):
        """Load data for retriever"""
        self.retriever.load_data(claims_path, evidence_path)

    def load_dev_data(self, dev_claims_path):
        """Load dev data for retriever"""
        self.retriever.load_dev_data(dev_claims_path)

    # def train(self, output_dir='./trained_model'):
    #     """Train the retriever"""
    #     self.retriever.train(output_dir)

    def index_evidence(self, save_path='data/evidence_embeddings.pth'):
        """Index evidence for retriever"""
        self.retriever.index_evidence(save_path)
    def get_evidence_text(self, evidences):
        eids = [eid for eid, _ in evidences]
        return [(eid, self.retriever.evidence_map[eid]) for eid in eids]
    def retrieve_and_filter(self, claim_text, top_k_initial=10, top_k_final=5):
        """Retrieve top 10 with sentence-transformers, filter to top 5 with LLaMA"""
        initial_results = self.retriever.retrieve(claim_text, top_k=top_k_initial)
        final_results = self.filter.filter_evidence(claim_text, get_evidence_text(initial_results), final_k=top_k_final)
        return final_results

    def evaluate(self, claims_path, top_k_initial=10, top_k_final=5):
        """Evaluate the hybrid system"""
        with open(claims_path) as f:
            claims = json.load(f)

        recall_initial = 0
        recall_final = 0
        precision_final = 0
        mrr_final = 0
        runtimes = []
        memory_usages = []

        import time
        import psutil
        process = psutil.Process(os.getpid())

        for claim_id, claim_data in claims.items():
            claim_text = claim_data['claim_text']
            relevant = set(claim_data['evidences'])

            # Measure runtime and memory
            start_time = time.time()
            results = self.retrieve_and_filter(claim_text, top_k_initial, top_k_final)
            runtime = time.time() - start_time
            runtimes.append(runtime)
            memory_usages.append(process.memory_info().rss / 1024**2)  # MB

            # Get retrieved evidence IDs
            retrieved_final = {eid for eid, _, _ in results}
            retrieved_initial = {eid for eid, _ in self.retriever.retrieve(claim_text, top_k_initial)}

            # Recall for initial retriever
            recall_initial += len(relevant & retrieved_initial) / len(relevant) if relevant else 0

            # Recall for final hybrid
            recall_final += len(relevant & retrieved_final) / len(relevant) if relevant else 0

            # Precision for final hybrid
            precision_final += len(relevant & retrieved_final) / top_k_final if relevant else 0

            # MRR for final hybrid
            for rank, (eid, _, _) in enumerate(results, 1):
                if eid in relevant:
                    mrr_final += 1 / rank
                    break
            else:
                mrr_final += 0

        n = len(claims)
        return {
            "Recall@10 (Initial)": recall_initial / n,
            "Recall@5 (Final)": recall_final / n,
            "Precision@5 (Final)": precision_final / n,
            "MRR@5 (Final)": mrr_final / n,
            "Avg Runtime (s)": sum(runtimes) / n,
            "Avg Memory (MB)": sum(memory_usages) / n
        }